## A First Step in Understanding Quantum Computing

One of the enigmas of quantum mechanics is its element of chance and probability. Quantum computers can use that fact to generate better random numbers. It also serves as a very first baby step in quantum computing in general!

This is a simple example showing how to simulate a quantum random number generator *from scratch* in the [Julia](https://julialang.org/) programming language. Of cource you need a real quantum computer to *do* quantum computing so the objective here is just to understand how it works with simple code. For that reason this doesn't use a library for quantum computing, or any library for that matter, but rather aims to be the simplest possible tutorial on quantum computing concepts for people with some programming experience.

No knowledge of quantum computing is assumed and the maths is at a minimum but not zero. You do need to understand a little bit of matrix multiplication and what a complex number is (although that'll be later and mostly hidden). The Julia syntax is very clean and will be easy to follow for most programmers. I admit that the unicode variable names are a bit experimental but I hope that they help get the message across.

This is not a popular science explanation and we'll get into some actual code which simulates what an actual quantum computer does. Let's get started.

### Quantum bits

We'll begin by defining a symbol, let's say ⬆, to represent the value `true` or `1`, a single quantum bit or qubit. We could have chosen any symbol and actually `|1>` is most often used in the field but that's harder to put in code and the significance of the arrow will be seen in a second. 

The definition will be a combination of **two** numbers, one of the numbers representing its *upness* and the other its *downness*.

In [1]:
const ⬆ = [1., 0.]; # Full upness and no downness

The **const** keyword tells us that the value cannot be changed, it's a definition. Likewise we can define a second symbol, not surprisingly ⬇, to represent `false` or `0`.

In [2]:
const ⬇ = [0., 1.]; # No upness and full downness

### Gates

What can we do with these symbols? Well, if we think about them as normal binary bits for a moment, `true` and `false`, what's the most basic thing we could do with them? 

One thing we could do is to flip its value with a NOT gate. We can do the same thing with our arrows by multiplying them by a 2x2 matrix (this is where the linear algebra comes in). We define the [quantum NOT gate](https://en.wikipedia.org/wiki/Quantum_logic_gate#Pauli-X_gate) (also called an X-gate) as a matrix multiplication like this and check that it does indeed flip our qubits.

In [3]:
const NOT = [0. 1.
             1. 0.]

@assert NOT*⬆ ≈ ⬇
@assert NOT*⬇ ≈ ⬆

(the **@assert** command just checks that the condition is true, if it doesn't output an error then we're happy :)

Rather than a flip, we can also consider this a *rotation* of 180º. Then you might ask yourself, can we modify that matrix to rotate the arrow by other amounts, say 90º instead of 180º? 

The answer is: we can. It's the matrix that when multiplied twice would give us our 180º rotation. It's the [square root of NOT](https://en.wikipedia.org/wiki/Quantum_logic_gate#Square_root_of_NOT_gate_(%E2%88%9ANOT))!

In [4]:
@assert √NOT * √NOT ≈ NOT
@assert √NOT * √NOT * ⬆ ≈ ⬇

If we rotate the value ⬆ (`true`) by 180º it becomes ⬇ (`false`) and vice versa, but if we rotate it by 90º it will become some new, intermediate value, lets say ⬅, that's neither `true` nor `false` but something in between. 

So what does that mean? Does it mean anything at all? 

It could be the answer to the question: Is Schrödinger's [cat](https://en.wikipedia.org/wiki/Schr%C3%B6dinger%27s_cat) alive? 🐈Neither yes nor no but a bit of both. This is called *superposition* and is key to quantum computing. It's physics in code.

And what's more, combining powers (like the square root, cube root, etc.) of these gates can give us any rotation we want!

### Measurement

But (and it's a big but!) the strange thing about these intermediate values is that **we can't see them, ever**. If we try then we'll always get ⬆ or ⬇, for sure, nothing in between. It *collapses*. The closer our rotated arrow is to ⬆, the more likely we'll see that value but it's a matter of chance which one we actually see. This weird fact comes straight out of quantum mechanics and is the source of randomness in a real quantum random number generator.

So how do we observe, or *measure* the thing anyway? In the case of a single qubit its just the amount of *upness* in the bit.

The following code *simulates* this measurement, using the `rand` function as a source of randomness. On a real quantum computer the measurement and the randomness is part of the quantum nature of the system itself. You can consider this function a black-box and skip the details.

In [5]:
measure(q) = abs(q[1])^2 > rand();

As an example, the probability of ⬆ being `true` should obviously be 100%, right? Likewise the probability of ⬇ being `true` should be 0%. Let's check.

In [6]:
@assert count(measure(⬆) for i in 1:100) == 100
@assert count(measure(⬇) for i in 1:100) == 0

Now, what happens if we measure our intermediate state, ⬅, 100 times? Let's try it.

In [7]:
const ⬅ = √NOT * ⬇;

In [8]:
count(measure(⬅) for i in 1:100)

49

In fact, on average It'll be about a 50% chance of either ⬇ or ⬆!

### Generating a random number

Now we can generate any random number we want by generating multiple bits this way. Just as an example, the number from 0 to 7 can be generated by converting 3 binary bits to an integer.

In [9]:
sum(2^i*measure(⬅) for i in 0:2)

4

### Wrap up

How's that for a first baby step in quantum computing? What have we covered?

- Quantum bits, or *qubits*, can represent `true` and `false` but also a whole continuum of intermediate values formed by rotations of any angle. These intermediate values are called *superpositions* and can never actually be seen.
- When *measured*, qubits always *collapse* to `true` and `false` and they do that in a specific but random way.

This is just a taster of what's to come. For example, the bits can actually rotate through more than one angle! They can also get *entangled*, another quantum mystery but amazingly useful for the quantum computer. For that we'll need combine multiple qubits and work on them together. Quantum mechanics tells us how to do that too and we'll cover it next...

<hr>

We could have used any of many of the existing existing quantum programming libraries for this but I wanted to start from scratch and Julia is literally the best language to do that in. If you want to get deeper into the other implementations here are some resources for futher study:
    
[Create a Quantum Random Number Generator](https://docs.microsoft.com/en-us/quantum/quickstarts/qrng) - Microsoft's version of this code written in their quatum computing language Q#.

[Truly Quantum Dice](https://medium.com/design-ibm/truly-quantum-dice-cfe372f4c586) - Some similar code written with IBM's [Qiskit](https://qiskit.org/).